# CMF Computation Tool

In [1]:
import pandas as pd

In [2]:
crash_data = pd.read_excel('D6 Crash Data.xlsx')

# Show column names and data types
for idx in range(len(crash_data.columns)):
    print(crash_data.columns[idx], '\t' ,crash_data.dtypes[idx])


FID 	 int64
Shape 	 object
OBJECTID 	 int64
OBJECTID__ 	 int64
Shape__ 	 object
Collision_ 	 int64
Agency_Nam 	 object
Date_and_T 	 datetime64[ns]
Area__Coun 	 object
Road_Name 	 object
Intersecti 	 object
KABCO_Seve 	 object
X__of_Fata 	 int64
X__Serious 	 int64
X__Visible 	 int64
Manner_of_ 	 object
Location_a 	 object
First_Harm 	 object
Most_Harmf 	 object
Operator__ 	 object
X__of_Vehi 	 int64
Vehicle_Ma 	 object
Direction_ 	 object
Light_Cond 	 object
Surface_Co 	 object
Driver_Age 	 int64
Safety_Equ 	 object
Vehicle_Ty 	 object
Inventory_ 	 object
MilePoint 	 float64
Latitude 	 float64
Longitude 	 float64
OBJECTID_1 	 int64
OBJECTID_2 	 int64
OBJECTID_3 	 int64
c_segid 	 object
c_sectid 	 int64
c_id 	 int64
c_type 	 object
c_radius 	 float64
c_devangle 	 float64
c_length 	 float64
c_pc_x 	 float64
c_pc_y 	 float64
c_pt_x 	 float64
c_pt_y 	 float64
g_rcd 	 object
g_rte_typ 	 int64
g_rte_cod 	 int64
g_rte_numb 	 int64
g_rte_suff 	 object
g_spd_limt 	 int64
p_bbi_max 	 float64
p_bb

## Pre-process crash data

In [3]:
# function for determine relation to treatment
def relation_to_treatment(data_year: int, treatment_year: int = 2016, exclude_years: list = [2020]):
    if data_year == treatment_year or data_year in exclude_years:
        return "unknown"
    elif data_year < treatment_year:
        return "before treatment"
    elif data_year > treatment_year:
        return "after treatment"


In [4]:
# pre-processing

# assign OBJECTID behind 'Longitude' as curve_id
crash_data['curve_id'] = crash_data.iloc[:, crash_data.columns.get_loc('Longitude')+1]
crash_data['year'] = crash_data.apply(lambda row: row['Date_and_T'].year, axis=1)

# relation to treatment excluding covid year
crash_data['Relation_To_HFST_Treatment_No_Covid'] = crash_data.apply(lambda row: relation_to_treatment(row['year'], treatment_year=2016, exclude_years=[2020]), axis=1)
# relation to treatment including covid year
crash_data['Relation_To_HFST_Treatment'] = crash_data.apply(lambda row: relation_to_treatment(row['year'], treatment_year=2016, exclude_years=[]), axis=1)
# single vehicle crash
crash_data['Single_Vehicle'] = crash_data.apply(lambda row: row['X__of_Vehi'] == 1, axis=1)
# intersection related
crash_data['Intersection_Related'] = crash_data.apply(lambda row: "Turning" in str(row['Vehicle_Ma']), axis=1)
# motorcycle related
crash_data['Motorcycle_Related'] = crash_data.apply(lambda row: "Motorcycle" in str(row['Vehicle_Ty']), axis=1)



## Compute CMFs

In [5]:
def get_pivot_counts(data: pd.DataFrame, index='curve_id', field = 'Relation_To_HFST_Treatment_No_Covid', unique_values = ['before treatment', 'after treatment', 'unknown']):
    pivot_count = pd.DataFrame()
    for value in unique_values:
        pivot_count[value] = pd.pivot_table(
            data=data,
            index=index,
            values=[field],
            aggfunc={
                field: lambda column: column[column == value].count(),
            }
        )
    pivot_count['Grand Total'] = pivot_count.sum(axis=1)
    return pivot_count


def compute_cmf(pivot_table: pd.DataFrame, years_before_treatment=3, years_after_treatment=3, before_columne_name='before treatment', after_column_name='after treatment'):
    freq_before = pivot_table[before_columne_name].sum(axis=0)/years_before_treatment
    freq_after = pivot_table[after_column_name].sum(axis=0)/years_after_treatment
    if freq_before == 0:
        return 1
    return freq_after/freq_before


### Total CMF

In [6]:
# pivot table
total_cmf_table = get_pivot_counts(data=crash_data, field='Relation_To_HFST_Treatment_No_Covid')
total_cmf_table


,before treatment,after treatment,unknown,Grand Total
curve_id,,,,
1,2,1,1,4
2,3,1,0,4
3,1,0,0,1
4,0,0,1,1
6,2,1,2,5
...,...,...,...,...
269,1,0,1,2
270,2,0,0,2
271,0,3,0,3


In [7]:
# cmf results
total_cmf_table = get_pivot_counts(data=crash_data, field='Relation_To_HFST_Treatment_No_Covid')
total_cmf = compute_cmf(total_cmf_table, years_after_treatment=3, years_before_treatment=3)
print("total cmf without covid data: ", total_cmf.round(2))

total_cmf_table = get_pivot_counts(data=crash_data, field='Relation_To_HFST_Treatment')
total_cmf = compute_cmf(total_cmf_table, years_after_treatment=4, years_before_treatment=3)
print("total cmf with covid data: ", total_cmf.round(2))


total cmf without covid data:  0.7
total cmf with covid data:  0.69


### Single Vehicle CMF


In [8]:
# cmf results
filtered_data = crash_data[crash_data['Single_Vehicle']]
pivot_table = get_pivot_counts(data=filtered_data, field='Relation_To_HFST_Treatment_No_Covid')
cmf = compute_cmf(pivot_table, years_after_treatment=3, years_before_treatment=3)
print("total cmf without covid data: ", cmf.round(2))

filtered_data = crash_data[crash_data['Single_Vehicle']]
pivot_table = get_pivot_counts(data=filtered_data, field='Relation_To_HFST_Treatment')
cmf = compute_cmf(pivot_table, years_after_treatment=4, years_before_treatment=3)
print("total cmf with covid data: ", cmf.round(2))


total cmf without covid data:  0.62
total cmf with covid data:  0.6


### Surface Condition

In [9]:
# cmf results
filtered_data = crash_data[crash_data['Surface_Co']=='Wet']
pivot_table = get_pivot_counts(data=filtered_data, field='Relation_To_HFST_Treatment_No_Covid')
cmf = compute_cmf(pivot_table, years_after_treatment=3, years_before_treatment=3)
print("total cmf without covid data: ", cmf.round(2))

filtered_data = crash_data[crash_data['Surface_Co']=='Wet']
pivot_table = get_pivot_counts(data=filtered_data, field='Relation_To_HFST_Treatment')
cmf = compute_cmf(pivot_table, years_after_treatment=4, years_before_treatment=3)
print("total cmf with covid data: ", cmf.round(2))


total cmf without covid data:  0.47
total cmf with covid data:  0.44
